This file contains code for my transfer-learning attempt. I use data for English handwriting publicly available at http://www.ee.surrey.ac.uk/CVSSP/demos/chars74k/. I build a CNN to recongize these characters and then save the model to disk. The transfered model built here didn't transfer well to F.G's handwriting character tiles, and so I abandoned this approach.

Imports

In [0]:
import os
import numpy as np
import keras
from skimage.io import imread
from skimage.transform import resize
from matplotlib import pyplot as plt
from random import seed, sample
from sklearn.svm import LinearSVC
from sklearn.grid_search import GridSearchCV
from pprint import pprint
from sklearn.metrics import accuracy_score
from random import randint
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense

Untar the EnglishHnd.tgz file from http://www.ee.surrey.ac.uk/CVSSP/demos/chars74k/

In [0]:
!tar -xvf EnglishHnd.tgz

Read the images and resize to 20 X 20 pixels. The character classes are - A-Z, a-z and 0-9. I map the upper and lower case letters to the same class, and hence have 36 classes in total.

In [0]:
rootDir = './English/Hnd/Img'
n_images = 6420
images = np.zeros((n_images,) + (20, 20))

labels = []

j = 0
for dirName, subdirList, fileList in os.walk(rootDir):
    i = 0
    if 'Sample' in dirName:
        label = int(dirName[-2:])
        if label > 36:
            label -= 26
    for fname in fileList:
        if 'png' not in fname:
            continue
        full_path = os.path.join(dirName, fname)
        images[j] = resize(imread(full_path, as_grey=True), (20, 20)) 
        labels.append(label)
        j += 1
        i += 1
images = images[:len(labels)]

Use a 80% - 20% split of training set - test set split

In [4]:
k = sample(range(len(images)), len(images))
im_shuf = images[k]
labels_shuf = np.array(labels)[k]
    
ocr = {
    'images': im_shuf,
    'data': im_shuf.reshape((im_shuf.shape[0], -1)),
    'target': labels_shuf
}

x_train, x_test, y_train, y_test = train_test_split(im_shuf, labels_shuf, random_state=2, train_size=0.8)

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [5]:
num_classes = 37 # Class 0 is ignored
batch_size = 128

X_train = x_train.reshape(x_train.shape[0], 20, 20 , 1).astype('float32')
X_test = x_test.reshape(x_test.shape[0], 20, 20 , 1).astype('float32')
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

model = Sequential()

model.add(Conv2D(32, kernel_size=(5, 5), activation='relu', input_shape=(20, 20, 1)))

model.add(Conv2D(64, (5, 5), activation='relu'))

model.add(Conv2D(128, (5, 5), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Dropout(0.25))
model.add(Flatten())
model.summary()
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])
model.fit(X_train, y_train, batch_size=batch_size,
                        nb_epoch=50,
                        verbose=1)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 16, 16, 32)        832       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 12, 12, 64)        51264     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 8, 8, 128)         204928    
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 4, 4, 128)         0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 4, 4, 128)         0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 2048)              0         
Total params: 257,024
Trainable params: 257,024
Non-trainable params: 0
_________________________________________________________________


/usr/local/lib/python3.6/dist-packages/keras/models.py:981: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Epoch 1/50
2728/2728 [==============================] - 10s 4ms/step - loss: 3.6034 - acc: 0.0286
Epoch 2/50
2728/2728 [==============================] - 10s 4ms/step - loss: 3.5687 - acc: 0.0385
Epoch 3/50
2728/2728 [==============================] - 10s 4ms/step - loss: 3.5130 - acc: 0.0590
Epoch 4/50
2728/2728 [==============================] - 10s 4ms/step - loss: 3.3333 - acc: 0.1001
Epoch 5/50
2728/2728 [==============================] - 10s 4ms/step - loss: 3.0886 - acc: 0.1503
Epoch 6/50
2728/2728 [==============================] - 10s 4ms/step - loss: 2.7932 - acc: 0.2163
Epoch 7/50
2728/2728 [==============================] - 10s 4ms/step - loss: 2.5532 - acc: 0.2680
Epoch 8/50
2728/2728 [==============================] - 10s 4ms/step - loss: 2.2825 - acc: 0.3457
Epoch 9/50
2728/2728 [==============================] - 10s 4ms/step - loss: 2.0707 - acc: 0.3970
Epoch 10/50
2728/2728 [==============================] - 10s 4ms/step - loss: 1.9144 - acc: 0.4326
Epoch 11/50
2728/27

In [6]:
scores = model.evaluate(X_test, y_test, verbose = 10 )
print ( scores )

[1.2553904509264702, 0.7023460407061312]


In [7]:
# serialize model to JSON
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model.h5")
print("Saved model to disk")

Saved model to disk


In [0]:
ynew = model.predict_classes(X_test)